In [ ]:
# DATA SET CANCER DE MAMA https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Original%29

from __future__ import division
churn_df = pd.read_csv('churn-data.csv')
col_names = churn_df.columns.tolist()

print "Column names:"
print col_names

to_show = col_names[:6] + col_names[-6:]

print "\nSample data:"
churn_df[to_show].head(6)

In [ ]:
import numpy as np
# from sklearn import preprocessing, cross_validation, neighbors
import pandas as pd

df = pd.read_csv('churn-data_prototipo.csv')
df.head()

In [2]:
# preprocesamiento ELIMINAR CAMPOS
to_drop = ['State','Area_Code','Phone']
df.drop(to_drop, 1, inplace=True)


In [3]:
#PREPROCESAMIENTO
df.replace('?',-99999, inplace=True)
for col in df.columns:
    # esto equivale a eliminar la fila
    df = df[df[col] != -99999]
    

In [5]:
# info
print df.head()
print df.shape[0]

   Account_Length  Int_Plan  VMail_Plan  VMail_Message  Day_Mins  Day_Calls  \
0             128         0           1             25     265.1        110   
1             107         0           1             26     161.6        123   
2             137         0           0              0     243.4        114   
3              84         1           0              0     299.4         71   
4              75         1           0              0     166.7        113   

   Day_Charge  Eve_Mins  Eve_Calls  Eve_Charge  Night_Mins  Night_Calls  \
0       45.07     197.4         99       16.78       244.7           91   
1       27.47     195.5        103       16.62       254.4          103   
2       41.38     121.2        110       10.30       162.6          104   
3       50.90      61.9         88        5.26       196.9           89   
4       28.34     148.3        122       12.61       186.9          121   

   Night_Charge  Intl_Mins  Intl_Calls  Intl_Charge  CustServ_Calls  Churn

In [ ]:
print(df.columns)

In [ ]:
#print(df.values)

In [ ]:
#print(df.describe)

In [ ]:
#df['Class']    # Class es el Churn en este proyecto

In [ ]:
#df[0:2]

In [ ]:
#df.iloc[2]

In [ ]:
# Ponemos condiciones así
#df[df.Class > 3]

In [6]:
# CLASIFICADOR montamos los arrays de NUMPY. Modifico Class por Churn

X = np.array(df.drop(['Churn'], 1), dtype=int)   
y = np.array(df['Churn'], dtype=int)

### VARIOS CLASIFICADORES CON CROSSFOLDER 

In [7]:
# -VARIOS CLASIFICADORES SVC, RF, KNN, LOG_REG ----------------------------

from sklearn.cross_validation import KFold

#  definimos la funcion de cross validation
def run_cv(X,y,clf_class,**kwargs):
    # Construimos el objeto kfolds 
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()
    
    # tenemos  que iterar sobre todos los kfolds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        
        
        # Iniciamos el clasificador con los keywords arguments...
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LOG_REG

from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler



# ------- CONFIANZA, RECALL, PRECISION, ETC. ---------------
print "--------  Confianzas de los calsificadores: \n \n "
pred_SVC = run_cv(X,y,SVC)
print "Confianza de SVC = ", accuracy(y, pred_SVC)
print "MATRIZ DE CONFUSION de SVC \n", confusion_matrix(y, pred_SVC), "\n"

pred_KNN = run_cv(X,y,KNN)
print "Confianza de KNN = ", accuracy(y, pred_KNN)
print "MATRIZ DE CONFUSION de KNN \n", confusion_matrix(y, pred_KNN), "\n"

pred_LOG_REG = run_cv(X,y,LOG_REG)
print "Confianza de  Log. Regression = ", accuracy(y, pred_LOG_REG)
print "MATRIZ DE CONFUSION de Log. Regression \n", confusion_matrix(y, pred_LOG_REG), "\n"

#print "%.3f" % accuracy(y, run_cv(X,y,RF))
pred_RF = run_cv(X,y,RF)
print "Confianza de Random forest:", accuracy(y, pred_RF)
print "MATRIZ DE CONFUSION de RANDOM FOREST \n", confusion_matrix(y, pred_RF), "\n"

--------  Confianzas de los calsificadores: 
 
 
Confianza de SVC =  0.855085508551
MATRIZ DE CONFUSION de SVC 
[[2850    0]
 [ 483    0]] 

Confianza de KNN =  0.882388238824
MATRIZ DE CONFUSION de KNN 
[[2797   53]
 [ 339  144]] 

Confianza de  Log. Regression =  0.859585958596
MATRIZ DE CONFUSION de Log. Regression 
[[2781   69]
 [ 399   84]] 

Confianza de Random forest: 0.938793879388
MATRIZ DE CONFUSION de RANDOM FOREST 
[[2816   34]
 [ 170  313]] 



### Calculamos los limites de los SLIDES de BOKEH para que estén derentro del margen de los valores del data set

In [9]:
print np.amin(X,axis=0)
print np.amax(X,axis=0)

[ 1  0  0  0  0  0  0  0  0  0 23 33  1  0  0  0  0]
[243   1   1  51 350 165  59 363 170  30 395 175  17  20  20   5   9]


In [ ]:
# RECORREMOS AHORA TODO EL DATAFRAME DE FORMA QUE SEA APLICABLE A CUALQUIERA DE LOS DATA FRAMES DE LOS CASOS... 

lista_minimos_col = []
lista_maximos_col = []

# recorremos las columnas del DATA FRAME
for i in range (df.shape[1]-1):                  # EL -1 es porque exxluyo el ultimo campo de la categoria o label 
    #print df.iloc[1][i:1]
    #print df.values[1][i]
    # añadimos el MIN y MAXde esa columna del dataframe
    lista_minimos_col.append(df.ix[df[df.columns[i]].idxmin()][df.columns[i]])
    lista_maximos_col.append(df.ix[df[df.columns[i]].idxmax()][df.columns[i]])

print lista_minimos_col
print lista_maximos_col

# Ahora calculamos los puntos cercanos a un punto dado para calcular donde están mas proximas los puntos de categoria diferente, (cancerosas). Esto nos ayudará a seleccionar las direcciones más relevantes a la hora de estudiar la evolución del punto en cuestión.
##UTLIZAMOS UN EJEMPLO SENCILLO DE DATAFRAME EN SU LUGAR

In [13]:
# en este caso de CHURN la ETIQUETA de clase es binaria, churn es 1, no-churn = 0
print X[y==0].shape
print X[y==1].shape

(2850L, 17L)
(483L, 17L)


In [14]:
classes = {}
for each in np.unique(y).tolist():
    classes[each] = X[y==each]

In [15]:
classes = {}
for each in np.unique(y).tolist():
    classes[each] = X[y==each]   

no_churn = classes[labels['no_churn']]   # (0´s, no_churn sustituye a benign)
churn = classes[labels['churn']]

b_dims = no_churn.shape
m_dims = churn.shape

no_churn = np.reshape(no_churn, b_dims + (1,))
churn = np.reshape(churn, m_dims + (1,))

np.tile(no_churn, [1, 1, m_dims[0]]) - 

SyntaxError: invalid syntax (<ipython-input-15-6eb51492b2c8>, line 14)

# PREDICCIONES CON INPUTS

In [ ]:
# vamos a hacer predicciones :

example_measures = np.array([[10,5,5,3,6,7,7,10,1]])
example_measures = example_measures.reshape(len(example_measures), -1)
prediction = clf.predict(example_measures)
print(prediction)

In [ ]:
print(df.values)

# 2 PARTE: gráficas-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
# condiciones que definen nuevos datasets
df_tumoral = df[df.Class > 3]
df_NO_tumoral = df[df.Class < 3]

df_tumoral.head()

In [ ]:
# graficas scattered 
import matplotlib.pyplot as plt
#scattercols = ['price','accommodates', 'number_of_reviews', 'reviews_per_month', 'beds', 'availability_30', 'review_scores_rating']
scattercols = df.columns
axs = pd.scatter_matrix(df[scattercols],
                        figsize=(12, 12), c='red')

plt.figure(figsize=(1920,1080))
plt.show()

In [ ]:
# definimos una funcion distancia de un registro del data se

In [ ]:
df_NO_tumoral.head()

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')

x = df_tumoral.Clump_Thickness
y = df_tumoral.Uniform_Cell_Shape
z = df_tumoral.Unifom_Cell_Size

x2 = df_NO_tumoral.Clump_Thickness
y2 = df_NO_tumoral.Uniform_Cell_Shape
z2 = df_NO_tumoral.Unifom_Cell_Size


ax1.scatter(x, y, z, c='g', marker='o')
ax1.scatter(x2, y2, z2, c ='r', marker='o')

ax1.set_xlabel('Espesor')
ax1.set_ylabel('Forma Celular')
ax1.set_zlabel(u'Tamaño Celular')

plt.show()

In [ ]:
#imprimimos las 3 primeras variables de prueba

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')



x = df_tumoral.eval(df_tumoral.columns[1])
y =  df_tumoral.eval(df_tumoral.columns[2])
z =  df_tumoral.eval(df_tumoral.columns[3])

x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[1])
y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[2])
z2 = df_NO_tumoral.eval(df_NO_tumoral.columns[3])


ax1.scatter(x, y, z, c='g', marker='o')
ax1.scatter(x2, y2, z2, c ='r', marker='o')

ax1.set_xlabel('Espesor')
ax1.set_ylabel('Forma Celular')
ax1.set_zlabel(u'Tamaño Celular')

plt.show()

In [ ]:
print(df.columns)

In [ ]:
print(df.columns[9])
print(len(df.columns))
      

In [ ]:
# funcion GENERICA QUE DIBUJA UNA COMBINACION particulr de las variable

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

def dibuja_grafica(i,j,k):

    style.use('ggplot')

    fig = plt.figure()
    ax1 = fig.add_subplot(111, projection='3d')


    x = df_tumoral.eval(df_tumoral.columns[i])
    y =  df_tumoral.eval(df_tumoral.columns[j])
    z =  df_tumoral.eval(df_tumoral.columns[k])

    x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[i])
    y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[j])
    z2 = df_NO_tumoral.eval(df_NO_tumoral.columns[k])

    ax1.scatter(x, y, z, c='g', label='TUMORALES', marker='o')
    ax1.scatter(x2, y2, z2, c ='r', label='SANAS', marker='o')
    
    #plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
    #plt.scatter(x2,y2, label='SANAS', color='r', s=25, marker="o")
 
    ax1.set_xlabel(df_tumoral.columns[i])
    ax1.set_ylabel(df_tumoral.columns[j])
    ax1.set_zlabel(df_tumoral.columns[k])
    
    plt.title('Breast Cancer Dataset Analysis               ')
    plt.legend()

    plt.show()  
    #plt.savefig('graf_'+df_tumoral.columns[i]+df_tumoral.columns[j]+df_tumoral.columns[k]+'.png')

dibuja_grafica(1,2,4)
    
    

    

In [ ]:
df.columns[-1]

In [ ]:
# representacion de todas las GRAFICAS combinacion total de variables

for counter1 in range(0,len(df.columns)-4):
    for counter2 in range(counter1 + 1,len(df.columns)):
        for counter3 in range(counter2 + 1,len(df.columns)):
            try:
                dibuja_grafica(counter1,counter2,counter3)
                #print counter1,",", counter2,",", counter3
            except Exception:
                pass

In [ ]:
# recordatorio de graficas en 2D

import matplotlib.pyplot as plt

x = [1,2,3,4,5,6,7,8]
y = [5,2,4,2,1,4,5,2]

x2 = [2,3,3,7,5,2,5]
y2 = [3,1,0,2,4,5,2]


#style.use('ggplot')

fig = plt.figure() 

# AÑADIMOS UN SUBPLOT sobre fig
ax1 = fig.add_subplot(111)

ax1.scatter(x, y, c='g', marker='o')
ax1.scatter(x2, y2, c ='r', marker='o')

plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
plt.scatter(x2,y2, label='NORMALES', color='r', s=25, marker="o")

plt.xlabel('x')
plt.ylabel('y')
plt.title('Breast Cancer Analysis')
plt.legend()
plt.show()

In [ ]:
# GRAFICA EN 2D  con la clase como Z
import matplotlib.pyplot as plt
from matplotlib import style

def dibuja_grafica2D(i,j,k):

    style.use('ggplot')
    
    x = df_tumoral.eval(df_tumoral.columns[i])
    y =  df_tumoral.eval(df_tumoral.columns[j])

    x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[i])
    y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[j])
    
    fig = plt.figure() 

    # AÑADIMOS UN SUBPLOT sobre fig
    ax1 = fig.add_subplot(111)

    ax1.scatter(x, y, c='g', marker='o')
    ax1.scatter(x2, y2, c ='r', marker='o')

    plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
    plt.scatter(x2,y2, label='SANAS', color='r', s=25, marker="o")

    # ax1.scatter(x, y, z, c='g', marker='o')
    # ax1.scatter(x2, y2, z2, c ='r', marker='o')

    ax1.set_xlabel(df_tumoral.columns[i])
    ax1.set_ylabel(df_tumoral.columns[j])


    plt.title('Breast Cancer Dataset Analysis - Human Forecast')
    plt.legend()
    #plt.show()
    plt.savefig('graficos/graf_'+df_tumoral.columns[i]+df_tumoral.columns[j]+'.png')


dibuja_grafica2D(1,2,9)

In [ ]:
# pero ahora solo estamos interesados en combinar dos a dos 
# con z que sea la clase de tumor que es el indice 9

for counter1 in range(0,len(df.columns)-2):
    for counter2 in range(counter1 + 1,len(df.columns)-1):
            try:
                dibuja_grafica2D(counter1,counter2,9)
                #print counter1,",", counter2
            except Exception:
                pass